# Smoothing Accelerometer/IMU data!

## Interactive `bqplot` demo 😊

In [14]:
import numpy as np
import pandas as pd
import datetime
import bqplot.pyplot as plt
import bqplot as bq
import ipywidgets
import warnings


warnings.filterwarnings("ignore") # ;)

file_path = "./data/Motion_Data_stage1.csv"
df = pd.read_csv(file_path,)

print("File Read")

File Read


In [15]:
sec = 100        # 100 samples per second 
num_secs = 4096    # how big of a window we want to view

start = 20 * sec
end = start + (num_secs * sec)

# iloc allows you to slice from and/or locate entries by integer indices.
x = df['time'].iloc[start:end]
y = [df.iloc[start:end]['accelerometer_x'],
    df.iloc[start:end]['accelerometer_y'],
    df.iloc[start:end]['accelerometer_z'],
    df.iloc[start:end]['gyroscope_x'],
    df.iloc[start:end]['gyroscope_y'],
    df.iloc[start:end]['gyroscope_z']]

        
print(f"Length of data (s):  {df.size/sec}")
print(f"Length of selected (s): {(end-start)/sec}")

Length of data (s):  9592.0
Length of selected (s): 4096.0


In [16]:
f = plt.figure(title="acceleration, see how spikey?")
plt.plot(y[1],colors=['green'])
f.marks[0].scales['x'].max=15000
f.marks[0].scales['x'].min=14000
f.marks[0].scales['y'].max=200
f.marks[0].scales['y'].min=-200
plt.show()

## Median Filtering the spikes out! 😀

A **median filter** "squashes" sudden impulses within a kernel size, or window, down to the median. The wider the kernel size, the greater the width of the impulses that can be "squashed." Too big of a window will eventually "squash" away the signal we wish to keep.

In the spirit of interactivity, I made a widget to interact with this `kernel_size` argument that the `medfilt` function takes. I found the best window size to be about 11-15 for the accelerometer in the y direction. You can also apply the median filter to the 3D or 6D array, but I would tune it to specific windows for acceleration and gyroscopic measurements.

> 💡 The high frequency spikes are likely the impulses from sudden changes in motion or direction, like when the foot hits the ground or pushes off, or in the case of running, even when the foot reverses direction mid air.
>
> You can imagine a ball in a box shaking and hitting different sides inside of a cube. Even though the direction of the box changed relatively smoothly, there will likely be shaking of the ball inside the box before it begins to accelerate along with the box, causing these spikes or impulses.

In [17]:
from scipy.signal import medfilt
from ipywidgets import interact, IntSlider
 
scales = {"x": bq.LinearScale(), "y": bq.LinearScale()}
xax = bq.Axis(scale=scales['x'])
yax = bq.Axis(scale=scales['y'],orientation='vertical')
line = bq.Lines(x=x,y=y[1],scales=scales, labels=['unfiltered'],opacities=[0.65],colors=['lime'])

filt_line = bq.Lines(x=x,y=medfilt(y[1]),scales=scales,labels = ['filtered line'], colors=['red'],preserve_axis=True)

@interact(kern_size=IntSlider(min=1,step=2,value=3,description="Kernel Window: (Must use odd number)"))
def ch_lin(kern_size):
    filt_line.y = medfilt(y[1],kernel_size=kern_size)

line.scales['y'].max = 5
line.scales['y'].min = -5
line.scales['x'].max = 210
line.scales['x'].min = 200

fig = bq.Figure(title="filter_effect",
                marks=[line,filt_line],
                axes=[xax,yax],
                scales=scales,
                animation_duration = 2000)

fig


interactive(children=(IntSlider(value=3, description='Kernel Window: (Must use odd number)', min=1, step=2), O…

Figure(animation_duration=2000, axes=[Axis(scale=LinearScale(max=210.0, min=200.0)), Axis(orientation='vertica…

The best kernel window I found was in the 11-15 range, so I chose 11 to err closer toward the integrity of the original data. However, the gyroscope data needs a different approach for removing spikes, noise and drift. We forego that for now, and save it for another stage of preparation.

For now, we will filter the acceleration data and add the filtered columns to the dataframe and the `.csv` file.

In [18]:
acc_columns = ['accelerometer_x','accelerometer_y','accelerometer_z']

for col in acc_columns:
    new_col_name = 'filtered_' + col
    df[new_col_name] = medfilt(df[col],kernel_size=11)
    
print(df.columns)

Index(['timestamp', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z',
       'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'time',
       'filtered_accelerometer_x', 'filtered_accelerometer_y',
       'filtered_accelerometer_z'],
      dtype='object')


Let's reorder these columns... Starting to bug me.

In [19]:
df = df[['timestamp','time','accelerometer_x',
         'accelerometer_y','accelerometer_z',
        'gyroscope_x','gyroscope_y',
         'gyroscope_z','filtered_accelerometer_x',
         'filtered_accelerometer_y','filtered_accelerometer_z']]

print(df.columns)

Index(['timestamp', 'time', 'accelerometer_x', 'accelerometer_y',
       'accelerometer_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z',
       'filtered_accelerometer_x', 'filtered_accelerometer_y',
       'filtered_accelerometer_z'],
      dtype='object')


Now we can save this new updated `.csv` file

In [21]:
df.to_csv('./data/motion_data_filt_acc.csv',index=False)


In [22]:
df.columns

Index(['timestamp', 'time', 'accelerometer_x', 'accelerometer_y',
       'accelerometer_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z',
       'filtered_accelerometer_x', 'filtered_accelerometer_y',
       'filtered_accelerometer_z'],
      dtype='object')

In [23]:
# Check if it saved!
import os

os.path.isfile('./data/motion_data_filt_acc.csv')

True

In [9]:
# Could also uses shell too
!dir data

 Volume in drive D is Local Sata SSD
 Volume Serial Number is 8E00-DAA8

 Directory of D:\Desktop\Projects\python\jupyter-notebooks\GaitML\data

05/06/2021  12:07 PM    <DIR>          .
05/06/2021  12:07 PM    <DIR>          ..
02/12/2020  09:38 PM        12,269,968 Motion_data.csv
05/07/2021  04:45 PM        16,707,344 motion_data_filt_acc.csv
04/24/2021  11:59 AM        12,597,830 Motion_data_stage1.csv
02/09/2020  12:48 AM        15,172,900 TAS1F06180329 (2018-10-24)-IMU.csv
               4 File(s)     56,748,042 bytes
               2 Dir(s)  476,517,982,208 bytes free


Now we are done filtering the accelerometer spikes! We can try to take out more noise, but the data looks much better now. We will eventually perform FFT analyses to get useful information from the accelerometer data, possibly to select features for our classification algorithm.

The next immediate goal, however, is to perform FFT analysis on the gyroscope data to hopefully filter out noise and isolate important properties of the signal to use for feature selection.

